In [45]:
# Look The Note Below!!!

import numpy as np
import time
import copy
import queue

N = 16 # You should change the test size here !!!

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# Read the note First ! ! !
#### In all the code above, only two of the function usages are needed. One is is_valid(self, state), which is to determine if the current state is legal; the other is is_satisfy(self, state), which is to determine if the current board meets the win condition. 
#### The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
#### In the first line of the code you can see N = 5, this is the size of the test.
#### Then in the test_block that follows, render indicates whether to use a graphical interface representation, and then method indicates which bts are used(bts or improving_bts).

### Request 1: You should complete the function bts(problem). 
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. (**DDL: 3.31**)
### Request 2: You should complete the function improving_bts(problem). 
You can select one or more methods of the three methods below(Minimum Remaining Value, Least constraining value, Forward checking), but you should have a good performance **when N = 16 without GUI**. (**DDL: 4.07**)

# BTS: Backtracking search (Request 1, DDL: 3.31)

In [32]:
def bts(problem):
    action_stack = []
    if N >=4:
        while not problem.is_satisfy(action_stack):
          row = 0
          col = 0
          while row < N:
             while col < N:
                action_stack.append((row,col))
                if problem.is_valid(action_stack):
                    row=row+1
                    col=0
                    break
                else:
                    action_stack.pop()
                    col=col+1
                while col == N:
                    row=-1
                    last=action_stack.pop()
                    col=last[1]+1
    yield action_stack
        # You can know what yield means in CSDN~

# Improving BTS To DO (Request 2, DDL: 4.07)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [71]:
def improving_bts(problem):
    remain = []
    action = []
    picked = []
    for i in range(N):
        temp = []
        for j in range(N):
            temp.append((i, j))
        remain.append(temp)
    if N >= 4:
        remain_stack = []
        next = 0
        action_stack = next_state(remain , next)
        action_stack.reverse()
        for p in action_stack:
            action.append(p)
            n_remain = copy.deepcopy(remain)
            remain_stack.append(valid_pos(action, n_remain))
            action.pop()
        while True:
            action.append(action_stack.pop())
            remain = remain_stack.pop()
            picked.append(next)
            min = 0
            for r in range(N):
                if (len(remain[r]) < len(remain[min]) and len(remain[r]) != 0 or len(
                        remain[min]) == 0 and min != r) and r not in picked:
                    min = r
            next = min
            if len(next_state(remain, next)) == 0:
                if len(picked) == N:
                    break
                else:
                    next = 0
                    action.clear()
                    picked.clear()
            for p in next_state(remain, next):
                action_stack.append(p)
                action.append(p)
                n_remain = copy.deepcopy(remain)
                remain_stack.append(valid_pos(action, n_remain))
                action.pop()
    yield action


def next_state(remain, next):
    pos = []
    c = N * N
    action = []
    for t in remain[next]:
        action.append(t)
        cop = copy.deepcopy(remain)
        cost = constraint(action, cop)
        if c > cost:
            c = cost
        action.pop()
    for t in remain[next]:
        action.append(t)
        cop = copy.deepcopy(remain)
        if c == constraint(action, cop):
            pos.append(t)
        action.pop()
    return pos


def valid_pos(action, remain):
    delet_remain = []
    row = 0
    a = action[-1]
    while row < N:
        temp = []
        for r in remain[row]:
            if a[0] == r[0] or a[1] == r[1] or abs(r[0] - a[0]) == abs(a[1] - r[1]):
                temp.append(r)
        delet_remain.append(temp)
        row = row + 1
    for row in range(N):
        for r in delet_remain[row]:
            remain[row].remove(r)
    return remain


def constraint(action, remain):
    num1 = 0
    num2 = 0
    for r1 in remain:
        num1 += len(r1)
    for r2 in valid_pos(action, remain):
        num2 += len(r2)
    return num1 - num2




In [72]:

# test_block
n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
render = False # here to select GUI or not
method = improving_bts  # here to change your method; bts or improving_bts
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in method(p):
        pass
    p.print_state(actions)
    print(time.time() - start_time)


_________________________________
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
---------------------------------
3.321998119354248
